# Regresi Linier - Satu Luaran, Banyak Prediktor

Penggunaan regresi linier tentu bisa diperluas menjadi banyak prediktor, kita bisa menyebut ini sebagai _multivariate linear regression_ atau regresi linier multivariat. Perluasan ini direpresentasikan dengan persamaan di bawah ini dengan $p$ adalah jumlah prediktor/fitur yang terkait dalam dataset.

\begin{equation}
Y \approx \beta_0 + \beta_1 x_{:,1} + \beta_2 x_{:,2} + \dots + \beta_p x_{:,p}
\end{equation}

atau dalam bentuk vektor

\begin{equation}
Y \approx
\begin{bmatrix}
\beta_0 & \beta_1 & ... & \beta_p  
\end{bmatrix}
\begin{bmatrix}
1 \\ x_{:,1} \\ x_{:,2} \\ \vdots \\ x_{:,p} 
\end{bmatrix}
\end{equation}


Secara umum, konsep untuk memilih parameter $\beta$ masih sama dengan sebelumnya, yakni dengan meminimasi RSS. 
Sebelumnya, RSS memiliki formula:
\begin{equation}
RSS = \sum_{i=1}^n {(y_i - \hat{\beta_0} - \hat{\beta_1} {x_i})^2}
\end{equation}

Sekarang menjadi (kita anggap $x_0$ adalah 1)
\begin{equation}
RSS = \sum_{i=1}^n  {(y_i - \sum_{j=0}^p \hat{\beta_j} {x_{i,j}})^2}
\end{equation}

Hanya saja pertanyaan yang muncul adalah kombinasi prediktor manakah yang paling baik untuk menentukan prediksi terhadap variabel target? Mari kita lihat dengan contoh kasus dengan dataset Wine Quality

## Latihan: Penggunaan Regresi Linier Multivariat

Dalam contoh kasus ini kita akan menggunakan dataset Red Wine Quality yang bisa diambil di "Data\Regression\Wine Quality\winequality-red.csv"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("Data/Regression/Wine Quality/winequality-red.csv",delimiter=";")
data.head()

In [ ]:
print(data.shape)

Variabel target yang akan kita prediksi adalah quality. Meskipun tipe datanya adalah bilangan bulat (integer), kita masih bisa menerapkan regresi terhadap dataset ini dengan pembulatan pada akhirnya. Mari kita bagi dataset kita dengan acak sebanyak dengan perbandingan 75:25 data latih berbanding data uji, dan X dan Y bersesuaian. Untuk memastikan masing-masing dari peserta mendapatkan nilai acak yang sama, kita menggunakan nilai random seed dari numpy = 0.

<b>Prosedur Pengacakan</b>

np.random.rand(n) akan membangkitkan n nilai acak dari 0 hingga 1. Kita akan menyeleksi data yang memiliki nilai acak kurang dari dan sama dengan 0.75 sebagai data latih dan sisanya sebagai data uji.

In [ ]:
np.random.seed(0)

X = data.drop(['quality'],axis=1)
Y = data['quality']

random_value = np.random.rand(data.shape[0])
X_train = X[random_value<=0.75]
Y_train = Y[random_value<=0.75]
X_test = X[random_value>0.75]
Y_test = Y[random_value>0.75]

#Untuk memastikan dimensi dari data latih dan data uji sudah sesuai
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Sebagaimana kita ketahui, pembangunan model regresi linier sangat mudah dengan pustaka scikit learn. Kita akan mencoba melihat bagaimana regresi linier yang dibangun dengan dua variabel prediktor. Kita akan mencoba membangkitkan semua pasangan 2 prediktor yang mungkin terjadi dari 11 variabel prediktor yang tersedia dengan syntax itertools

In [ ]:
import itertools
pred_pairs = list(itertools.combinations(X.columns.values,2))

print("Jumlah pasangan variabel prediktor yang terjadi: %d" % len(pred_pairs))
print("10 pasangan pertama:")
for i in range(10):
    print(pred_pairs[i])

Ada 55 pasang prediktor, jumlah yang cukup banyak. Mari kita lihat bagaimana performa dari masing-masing pasangan tersebut dengan data latih dan data uji yang sudah kita bagi. Dengan adanya list yang dibuat dengan itertools, seleksi data akan lebih mudah. Kita akan tampung performa dari masing-masing pasangan dengan dataframe baru: performances. Performances akan berisi atribut pairs (yang menandakan model tersebut menggunakan prediktor apa saja), test MSE (menyimpan nilai pengujian model terhadap data uji), dan build time (menandakan waktu yang dibutuhkan untuk membangun model) akan dibahas dalam subbab berikutnya.

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import time

performances = pd.DataFrame(columns=['pairs','Test MSE','build time'])

for pairs in pred_pairs:
    linreg = linear_model.LinearRegression()
    t = time.time()
    linreg.fit(X_train[list(pairs)], Y_train)
    build_time = time.time() - t
    Y_pred = linreg.predict(X_test[list(pairs)])
    mse = mean_squared_error(Y_test, Y_pred)
    performances.loc[len(performances)] = np.array([pairs[0] + "-" + pairs[1], mse,build_time])

Manakah pasangan variabel yang menghasilkan prediksi paling baik? Gunakan fungsi sort_values dan head pada DataFrame untuk mengetahuinya

In [ ]:
performances.sort_values(by='Test MSE', ascending=True).head()

Dengan cara yang sama:
1. Membangkitkan pasangan variabel prediktor dengan itertools, namun dengan 5 variabel prediktor
2. Menganalisis model terbaik dari pasangan triplet variabel prediktor dengan test MSE terendah

Apakah semakin banyak variabel prediktor membantu model regresi linier multivariat untuk memprediksi target lebih baik?

In [ ]:
pred_quintets = None

print("Jumlah pasangan variabel prediktor yang terjadi: %d" % len(pred_quintets))
print("10 quintet pertama:")
for i in range(10):
    print(pred_quintets[i])

In [ ]:
pred_quintets = list(itertools.combinations(X.columns.values,5))

print("Jumlah pasangan variabel prediktor yang terjadi: %d" % len(pred_quintets))
print("10 quintet pertama:")
for i in range(10):
    print(pred_quintets[i])

In [ ]:
performances5 = pd.DataFrame(columns=['quintets','Test MSE','build time'])

for quintet in pred_quintets:
    linreg = linear_model.LinearRegression()
    t = time.time()
    linreg.fit(None, Y_train)
    build_time = time.time() - t
    Y_pred = linreg.predict(None)
    mse = mean_squared_error(Y_test, Y_pred)
    performances5.loc[len(performances5)] = np.array([quintet[0] + "-" + quintet[1]+ "-" + quintet[2]+ "-" + quintet[3]+ "-" + quintet[4], mse, build_time])

In [ ]:
performances5.sort_values(None).head()

Kemudian mari kita coba dengan 11 variabel prediktor.

In [ ]:
linreg = linear_model.LinearRegression()
t = time.time()
linreg.fit(X_train, Y_train)
build_time = time.time() - t
Y_pred = linreg.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
print(list(X_train.columns))
print("MSE: %.3lf" % mse)
print("Build time: %lf" % build_time)

----------------------

## Curse of Dimensionality

Cost yang digunakan untuk pembuatan model regresi linier multivariat cenderung berbanding lurus dengan jumlah variabel prediktor yang dilatih dan jumlah baris. Mari kita lihat perbandingan waktu komputasi pembuatan model dengan 2 variabel prediktor, 5 variabel prediktor, dan 11 variabel prediktor.

In [ ]:
performances['build time'].astype(np.float).mean()

In [ ]:
performances5['build time'].astype(np.float).mean()

In [ ]:
build_time

#### Catatan: Build time ini akan sangat bergantung pada kecepatan komputer Anda. Model linier cenderung sangat mudah dibangun dan tidak menghabiskan sumber daya yang sangat banyak, sehingga waktu pembangunan model bisa sangat singkat. Bisa jadi pembangunan model dari 2, 5, dan 11 variabel ini memiliki nilai yang cenderung tidak menimbulkan pola naik karena kekuatan komputer yang sangat tinggi.

Walaupun cenderung tidak signifikan, terjadi peningkatan dari waktu pembangunan model dari regresi linier dengan variabel yang terus bertambah. Hal ini juga akan berpengaruh terhadap performa dari regresi linier itu sendiri. Semakin banyak variabel prediktor, akan cenderung membuat model tersebut mengalami overfitting terhadap data yang diuji. Data yang ada di dunia bahkan bisa memiliki lebih banyak fitur, seperti citra atau teks. Sedemikian hingga, jumlah variabel harus bisa kita reduksi sebaik mungkin dengan pemilihan variabel yang baik. Gambar di bawah ini menyatakan hubungan jumlah prediktor dengan RSS pada dataset lain.

![Alt Text](img/3. rss-numpred.JPG)

Untuk mengatasinya, kita akan menggunakan salah satu konsep regularisasi yang akan dibahas di bagian 6.